In [ ]:
def count_fasta_samples(fasta_file_path):
    sample_count = 0
    with open(fasta_file_path, 'r') as fasta_file:
        for line in fasta_file:
            if line.startswith('>'): # and 'test' not in line.lower(): 
                sample_count += 1
    return sample_count

# Example usage
fasta_file_path = './data/deeploc_data (1).fasta' 
sample_count = count_fasta_samples(fasta_file_path)
print(f"Number of data samples: {sample_count}")


Number of data samples: 14004


In [ ]:
def count_fasta_samples(fasta_file_path):
    sample_count = 0
    excluded_samples = []  # List to store excluded sample headers
    with open(fasta_file_path, 'r') as fasta_file:
        for line in fasta_file:
            if line.startswith('>'):
                if 'test' not in line.lower():  # Excluding test samples
                    sample_count += 1
                else:
                    excluded_samples.append(line.strip())  # Collect excluded lines

    print(f"Excluded Samples: {len(excluded_samples)}")
    if excluded_samples:
        print("\nSome Excluded Sample Headers:")
        for header in excluded_samples[:10]:  # Print first 10 headers for example
            print(header)

    return sample_count

fasta_file_path = './data/deeploc_data (1).fasta' 
sample_count = count_fasta_samples(fasta_file_path)
print(f"Number of data samples: {sample_count}")


Excluded Samples: 2773

Some Excluded Sample Headers:
>Q9H400 Cell.membrane-M test
>P83456 Cell.membrane-M test
>Q9GL77 Cell.membrane-M test
>Q03445 Cell.membrane-M test
>P35525 Cell.membrane-M test
>P35523 Cell.membrane-M test
>M9MSG8 Cell.membrane-M test
>Q0VD83 Cell.membrane-M test
>Q86T26 Cell.membrane-M test
>Q5VWK5 Cell.membrane-M test
Number of data samples: 11231


In [11]:
import numpy as np
import pandas as pd

df = pd.read_csv('./data/Swissprot_Train_Validation_dataset.csv')

print(df.head())

   Unnamed: 0     ACC  Kingdom  Partition  Membrane  Cytoplasm  Nucleus  \
0           0  Q28165  Metazoa          4       0.0        1.0      1.0   
1           1  Q86U42  Metazoa          4       0.0        1.0      1.0   
2           2  Q0GA42  Metazoa          3       1.0        0.0      0.0   
3           3  P82349  Metazoa          1       1.0        1.0      0.0   
4           4  Q7L5N1  Metazoa          1       0.0        1.0      1.0   

   Extracellular  Cell membrane  Mitochondrion  Plastid  \
0            0.0            0.0            0.0      0.0   
1            0.0            0.0            0.0      0.0   
2            0.0            1.0            0.0      0.0   
3            0.0            1.0            0.0      0.0   
4            0.0            0.0            0.0      0.0   

   Endoplasmic reticulum  Lysosome/Vacuole  Golgi apparatus  Peroxisome  \
0                    0.0               0.0              0.0         0.0   
1                    0.0               0.0  

In [12]:
df.shape

(28303, 16)